Notes: 
- FSQR ratings need to be halved to standardize with yelp

In [1]:
import pandas as pd
import sqlite3

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [2]:
# load part 2 data
results_FSQR = pd.read_csv('../data/POI_results/pois_FSQR.csv')
results_YELP = pd.read_csv('../data/POI_results/pois_YELP.csv')

In [3]:
# validate load
print(results_FSQR.info())
results_YELP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  370 non-null    object 
 1   station_id    370 non-null    object 
 2   API           370 non-null    object 
 3   id            370 non-null    object 
 4   category      370 non-null    object 
 5   distance      370 non-null    int64  
 6   name          370 non-null    object 
 7   rating        216 non-null    float64
 8   ratings       216 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 26.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  370 non-null    object 
 1   station_id    370 non-null    object 
 2   API           370 non-null    object 
 3   id            370 non-null    o

In [4]:
# join part 2 data
palmaQuery = pd.concat([results_FSQR, results_YELP], ignore_index = True)

In [5]:
# validate join
palmaQuery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  740 non-null    object 
 1   station_id    740 non-null    object 
 2   API           740 non-null    object 
 3   id            740 non-null    object 
 4   category      740 non-null    object 
 5   distance      740 non-null    float64
 6   name          740 non-null    object 
 7   rating        586 non-null    float64
 8   ratings       586 non-null    float64
dtypes: float64(3), object(6)
memory usage: 52.2+ KB


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

Here is the plot generated for total ratings, from the Foursquare vs Yelp analysis:

![](../images/ratingsPerBikeStationPalma.png)

or click [here](../images/ratingsPerBikeStationPalma.png)

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [6]:
# table creation queries
createTable_networks = '''CREATE TABLE networks
(
    network_id  VARCHAR PRIMARY KEY,
    company     VARCHAR,
    name        VARCHAR,
    city        VARCHAR,
    country     VARCHAR,
    latitude    FLOAT,
    longitude   FLOAT
)'''
createTable_stations = '''CREATE TABLE stations
(
    station_id  VARCHAR PRIMARY KEY,
    network_id  VARCHAR,
    name        VARCHAR,
    bikes       INTEGER,
    latitude    FLOAT,
    longitude   FLOAT
)'''
createTable_palmaQuery = '''CREATE TABLE palmaQuery
(
    station_id    VARCHAR,
    station_name  VARCHAR,
    API           VARCHAR,
    id            VARCHAR,
    category      VARCHAR,
    distance      FLOAT,
    name          VARCHAR,
    rating        FLOAT,
    ratings       FLOAT
)'''

In [7]:
# open database connection
conn_str = "../data/Biketouring.db"

conn = sqlite3.connect(conn_str) 
c = conn.cursor()
print('connection established')

connection established


In [8]:
# drop tables with duplicate name
c.execute("DROP TABLE IF EXISTS networks;")
c.execute("DROP TABLE IF EXISTS stations;")
c.execute("DROP TABLE IF EXISTS palmaQuery;")

In [9]:
# create tables
c.execute(createTable_networks)
c.execute(createTable_stations)
c.execute(createTable_palmaQuery)

print('tables created')

tables created


In [10]:
networks = pd.read_csv('../data/networks_all.csv')
networks.to_sql('networks', conn, if_exists='replace', index=False)

stations = pd.read_csv('../data/Networks/palmaBikes.csv')
stations.to_sql('stations', conn, if_exists='replace', index=False)

palmaQuery.to_sql('palmaQuery', conn, if_exists='replace', index=False)

740

In [11]:
# commit and close
conn.commit()
conn.close()
print('import to database completed')

import to database completed
